In [5]:
import numpy as np

In [676]:
class Layer():
    
    def __init__(self,no_of_neurons,previous_layer=None,activation=None,name='def'):
        # number of neurons in current layer
        self.n = no_of_neurons
        # layer that sends input into this layer
        self.previous_layer = previous_layer
        # layer that recieves this layer's output
        self.next_layer = None
        if (self.previous_layer is not None):
            # sort the hierarchy
            previous_layer.next_layer = self
            # initialize the weights between the layers
            previous_layer.initialize_weights()
            # set the activation function
            self.set_activation(activation)
        else:
            # this is the input layer
            self.set_activation('none')
        # for testing purposes
        self.name = name
    
    def initialize_weights(self):
        if (self.next_layer is not None):
            # weights
            self.weights = np.random.normal(0,0.1,(self.n,self.next_layer.n))
            # bias separately
            self.bias = np.random.normal(0,0.1,self.next_layer.n)
    
    def set_activation(self,func='ReLU'):
        if (func=='ReLU'):
            # set ReLU and the derivative
            self.func = ReLU
            self.deriv = dReLU
        elif (func=='none'):
            self.func = lambda x:x
            self.deriv = lambda x:1

    def feed_forward(self,z):
        # remember z and a for later
        self.z = z
        self.a = self.func(z)
        if (self.next_layer is not None):
            return self.next_layer.feed_forward(self.a @ self.weights + self.bias)
        else:
            return self.a
    
    def update_weights(self,learning_rate,error):
        self.weights -= self.weights*learning_rate*error
        self.bias -= self.bias*learning_rate*error
        
    def back_propagate(self,y=None,learning_rate=0.00001):       
        if (self.next_layer is None):
            self.error = self.a - y
            self.delta = self.error * self.deriv(self.z)
        else:
            # removed the .T from weights
            self.error = self.weights @ self.next_layer.delta
            self.delta = self.error * self.deriv(self.z)
            for i in range (self.weights.shape[1]):
                self.weights[:,i] -= learning_rate * (self.a * self.next_layer.delta[i])
            for i in range(len(self.bias)):
                self.bias[i] -= learning_rate * self.next_layer.delta[i]
        if (self.previous_layer is not None):
            self.previous_layer.back_propagate(learning_rate=learning_rate)
        
def ReLU(x):
    # ReLU function
    return np.maximum(x,0)

def dReLU(x):
    x = x.copy()
    x[x<=0] = 0
    x[x>0] = 1
    return x    

In [677]:
a = Layer(2,name='a')

In [678]:
b = Layer(50,a,activation='ReLU',name='b')

In [679]:
c = Layer(6,b,activation='none',name='c')

In [711]:
for i in range(10000):
    x = np.random.random(size=2)
    y = np.array([x[0],x[1],x[0]+x[1],x[0]-x[1],x[0]*x[1],3*x[0]-1.6*x[1]])
    a.feed_forward(x)
    c.back_propagate(y,learning_rate=0.0001)

In [712]:
x = np.array([0.99,0.67])
y = np.array([x[0],x[1],x[0]+x[1],x[0]-x[1],x[0]*x[1],3*x[0]-1.6*x[1]])
(a.feed_forward(x) - y).mean()

-0.10164558748679357

In [713]:
a.feed_forward(x)

array([0.91615567, 0.53557707, 1.47709228, 0.3618328 , 0.46692129,
       1.83384736])

In [603]:
a.feed_forward(x)

array([0.99617295, 0.67346865, 1.66621645, 0.32224475, 0.57991196])